In this notebook, we present the constant interest rate-constant volatility model. Here we only go through the implementation and calibration of the model, and evaluation of the IRR. For the description of the model, refer to the report.

We start with importing a few libaries.

In [ ]:
import numpy as np
import pandas as pd
import time
from datetime import date
from datetime import timedelta
from dateutil.relativedelta import *
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats

Next we define a few functions to modify dates; ideally we would like to only have a time axis and not have the concept of days at all, but the data is provided on weekdays.

In [ ]:
def add_days(d,x):
	d=d+timedelta(days=x)
	t=0
	if (d.weekday()==5):
		d=d+timedelta(days=2)
		t=2
	if (d.weekday()==6):
		d=d+timedelta(days=1)
		t=1
	return d
	
def next_workday(d):
	if (d.weekday()!=5 and d.weekday!= 6):
		d=d+timedelta(days=1)
	if (d.weekday()==5):
		d=d+timedelta(days=2)
	if (d.weekday()==6):
		d=d+timedelta(days=1)
	return d

def previous_workday(d):
	if (d.weekday()!=5 and d.weekday!= 6):
		d=d+timedelta(days=-1)
	if (d.weekday()==5):
		d=d+timedelta(days=-1)
	if (d.weekday()==6):
		d=d+timedelta(days=-2)
	return d

def to_workday(d):
    if (d.weekday()==5):
        d=d+timedelta(days=2)
    if (d.weekday()==6):
        d=d+timedelta(days=1)
    return d

def add_months(d,x):
    d=d+relativedelta(months=x)
    if (d.weekday()==5):
        d=d+timedelta(days=2)
    if (d.weekday()==6):
        d=d+timedelta(days=1)
    return d

def add_years(d,x):
    return add_months(d,x*12)

Next, we import and clean up the market and fund data. Remeber to modify the input data file paths.

In [ ]:
market_data=pd.read_excel(r'C:\cygwin64\home\Christos\case\data.xls')

market_data.rename(columns={"Unnamed: 1":"Date"},inplace=True)
market_data.drop("Unnamed: 0",axis=1,inplace=True)

Spot={"EURUSD":[]}
Forward={"EURUSD":{},"GBPUSD":{},"EURGBP":{}}
Imp_vol={"EURUSD":{},"GBPUSD":{},"EURGBP":{}}
Yield={"EUR":{},"USD":{},"GBP":{}}
Swaption={"EUR":{},"USD":{},"GBP":{}}

for c in market_data.columns:
    if c[0:3]=='Dat':
        market_dates=market_data[c][7:].tolist()
    if c[0:3]=='Spo':
        Spot[market_data[c][0]]=market_data[c][7:].tolist()
        s="Spot "+market_data[c][0]
        market_data.rename(columns={c:s},inplace=True)
        market_data[s]=market_data[s].apply(pd.to_numeric,errors='coerce')
    if c[0:3]=='For':
        if market_data[c][1] in {"MAR","JUN","SEP","DEC"}:
            market_data.drop(c,axis=1,inplace=True)
        else:
            Forward[market_data[c][0]][market_data[c][1]]=market_data[c][7:].tolist()
            s="Forward "+market_data[c][0]+" "+market_data[c][1]
            market_data.rename(columns={c:s},inplace=True)
            market_data[s]=market_data[s].apply(pd.to_numeric,errors='coerce')
    if c[0:3]=='Imp':
        maturity=market_data[c][1]
        if maturity=='ON':
            T=1/252
        if 'W' in maturity:
            T=int(maturity[0:-1])/52
        if 'M' in maturity:
            T=int(maturity[0:-1])/12
        if 'Y' in maturity:
            T=int(maturity[0:-1])
        Imp_vol[market_data[c][3][0:6]][T]=market_data[c][7:].tolist()
        s="Implied Volatility "+market_data[c][3][0:6]+" "+market_data[c][1]
        market_data.rename(columns={c:s},inplace=True)
        market_data[s]=market_data[s].apply(pd.to_numeric,errors='coerce')
    if c[0:3]=='Yie':
        Yield[market_data[c][0]][market_data[c][1]]=market_data[c][7:].tolist()
        s="Yield curve "+market_data[c][0]+" "+market_data[c][1]
        market_data.rename(columns={c:s},inplace=True)
        market_data[s]=market_data[s].apply(pd.to_numeric,errors='coerce')
    if c[0:3]=='Swa':
        Swaption[market_data[c][0]][market_data[c][1]]=market_data[c][7:].tolist()
        s="Swaption "+market_data[c][0]+" "+market_data[c][1]
        market_data.rename(columns={c:s},inplace=True)
        market_data[s]=market_data[s].apply(pd.to_numeric,errors='coerce')

market_data.drop(range(7),axis=0,inplace=True)
#####################
fund_data=pd.read_excel(r'C:\cygwin64\home\Christos\case\fund.xls')
fund_dates=fund_data['Date'].tolist()
fund_USD=fund_data['USD flow'].tolist()
fund_Euro=fund_data['Euro flow'].tolist()
fund_GBP=fund_data['GBP flow'].tolist()
##########################
market={}
market["dates"]=market_dates
market["data"]=market_data
market["Spot"]=Spot
market["Forward"]=Forward
market["Imp_vol"]=Imp_vol
market["Yield"]=Yield
market["Swaption"]=Swaption

fund={}
fund["data"]=fund_data
fund["dates"]=fund_dates
fund["USD"]=fund_USD
fund["Euro"]=fund_Euro
fund["GBP"]=fund_GBP

We read the interest rates off the input data, and determine the drifts of the Brownian motions representing the exchange rates. Remember, given a domestic interest rate $r_d$ and a foreign interest rate $r_f$, we model the exchange rate as 
$$
dR(t)=(r_d-r_f)R(t)dt+\sigma R(t) dW(t).
$$

In [ ]:
r_USD=np.log((1+Yield["USD"]["3M"][-1]/100))
r_Euro=np.log((1+Yield["EUR"]["3M"][-1]/100))
r_GBP=np.log((1+Yield["GBP"]["3M"][-1]/100))
print("USD short rate: ",r_USD)
print("Euro short rate: ",r_Euro)
print("GBP short rate: ",r_GBP)

print("\nX-Y spot rate: how much of X one unit of Y can buy")
m_USD_Euro=r_USD-r_Euro
m_GBP_USD=r_GBP-r_USD
m_Euro_GBP=r_Euro-r_GBP

print("USD-Euro drift: ",m_USD_Euro)
print("USD-Euro spot rate:",Spot["EURUSD"][-1])

print("GBP-USD drift: ",m_GBP_USD)
print("GBP-USD spot rate:",1/Spot["GBPUSD"][-1])

print("Euro-GBP drift: ",m_Euro_GBP)
print("Euro-GBP spot rate:",1/Spot["EURGBP"][-1])

Since we know the drifts, we need to extract the volatilities. To do so, we calibrate our model to minimize the mean squared error between historic market prices and predicted prices. In principle, one observation of the price of a simple option is sufficient to extract the volatility of our model. However, since we have multiple observations, we need to extract the volatility that best explains these observations simultaneously.

For the fitting, we use gradient descent with a well-chosen initialization. The initial value is important due to the non-convexity of the objective function we want to minimize. Instead of implementing gradient descent explicitly, we use tensorflow's implementation because it includes automatic differentiation. As a result, the calibration is slower than it should ideally be; if performance was an issue we would have calculated the gradient explicitly and implemented gradient descent using it.

In [ ]:
def calibrate_vol(C,maturities,rd,rf):
	(m,n)=np.shape(C)
	x = tf.Variable(0.05, name='x', dtype=tf.float32)
	E=0

	for i in range(m):
		for j in range(n):
			T=maturities[i]
			d2=(rd-rf-x**2/2)*T/(x*np.sqrt(T))
			d1=d2+x*np.sqrt(T)
			price=tfp.distributions.Normal(0,1).cdf(d1)*np.exp(-rf*T)-tfp.distributions.Normal(0,1).cdf(d2)*np.exp(-rd*T) #Predicted option price
			E=E+(C[i][j]-price)**2/(m*n)#Squared error between historic and predicted price

#Having set up the error function, tensorflow takes care of differentiating and optimizing
	optimizer = tf.train.GradientDescentOptimizer(0.35)
	train = optimizer.minimize(E)
	init = tf.initialize_all_variables()
	def optimize():
		with tf.Session() as session:
			session.run(init)
			fx=session.run(E)
			fprev=0
			print("starting at", "x:", session.run(x), "MSE:", fx)
			step=0
			while np.abs(fx-fprev)>=10**(-10):
				fprev=fx
				session.run(train)
				fx=session.run(E)
				print("step", step, "x:", session.run(x), "MSE:", fx)
				step=step+1
			ret=session.run(x)
			session.close()
			return ret

	x_opt=optimize()
	return x_opt

Since we are given implied volatilities, we need to extract option prices for them, and use those prices to calibrate our model.

In [ ]:
def vanilla_atm_call_option(rd,rf,s,T):
	d2=(rd-rf-s**2/2)*T/(s*np.sqrt(T))
	d1=d2+s*np.sqrt(T)
	x=norm.cdf(d1)*np.exp(-rf*T)-norm.cdf(d2)*np.exp(-rd*T)
	return x

def sample_prices(imp_vol,maturities,rd,rf,m,n):
	S=np.zeros(shape=(m,n))
	C=np.zeros(shape=(m,n))
	maturities=list(imp_vol.keys())
	for i in range(m):
		for j in range(n):
			S[i][j]=imp_vol[maturities[i]][-j-1]
			C[i][j]=vanilla_atm_call_option(rd,rf,S[i][j]/100,maturities[i])
	return (C,S)

n=1 #We calibrate with respect to the prices observed in the n last days
maturities=list(Imp_vol['EURUSD'].keys())
m=len(maturities)-2 #We consider options with m different maturities; essentially, we disregard the 7y and 10y maturities. We have chosen a constant volatility model, but the volatilities of these maturities are so different than the rest of the data, that their inclusion will more likely contribute to our model being more inaccurate.

print("Calibrating USD-Euro volatility to minimize MSE of option prices in last ",n," day(s)")
(C_USD_Euro,S_USD_Euro)=sample_prices(Imp_vol['EURUSD'],maturities,r_USD,r_Euro,m,n)
s_USD_Euro=calibrate_vol(C_USD_Euro,maturities,r_USD,r_Euro)

print("Calibrating GBP-USD volatility to minimize MSE of option prices in last ",n," day(s)")
(C_GBP_USD,S_GBP_USD)=sample_prices(Imp_vol['GBPUSD'],maturities,r_GBP,r_USD,m,n)
s_GBP_USD=calibrate_vol(C_GBP_USD,maturities,r_GBP,r_USD)

print("Calibrating Euro-GBP volatility to minimize MSE of option prices in last ",n," day(s)")
(C_Euro_GBP,S_GBP_USD)=sample_prices(Imp_vol['EURGBP'],maturities,r_Euro,r_GBP,m,n)
s_Euro_GBP=calibrate_vol(C_Euro_GBP,maturities,r_Euro,r_GBP)



We can plot the MSE as a function of volatility to verify that we our volatility is indeed a local minimum.

In [ ]:
def MSE(rd,rf,s,C,maturities,m,n):
	E=0
	for i in range(m):
		for j in range(n):
			C_predicted=vanilla_atm_call_option(rd,rf,s,maturities[i])
			E=E+(C_predicted-C[i][j])**2/(m*n)
	return E

def plot_MSE(r_name,rd,rf,s,C,maturities,m,n):
	print(r_name," volatility: ",s)
	print("Plotting MSE of option prices VS volatility to verify:")
	gx=np.linspace(max(0.001,s-0.1),s+0.1,100)
	gy=MSE(rd,rf,gx,C,maturities,m,n)
	plt.plot(gx,gy,color='b')
	plt.plot(s,MSE(rd,rf,s,C,maturities,m,n),marker="x",color='r')
	plt.xlabel(str(r_name+" volatility"))
	plt.ylabel("MSE")
	plt.title("MSE of predicted vs historical option prices as a function of volatility\n For perspective, min/avg/max option prices are\n"+'{:.2e}'.format(np.min(C))+"/"+'{:.2e}'.format(np.mean(C))+"/"+'{:.2e}'.format(np.max(C))+"\n optimum MSE="+'{:.2e}'.format(MSE(rd,rf,s,C,maturities,m,n)))
	plt.show()

plot_MSE("USD-Euro",r_USD,r_Euro,s_USD_Euro,C_USD_Euro,maturities,m,n)
plot_MSE("GBP-USD",r_GBP,r_USD,s_GBP_USD,C_GBP_USD,maturities,m,n)
plot_MSE("Euro-GBP",r_Euro,r_GBP,s_Euro_GBP,C_Euro_GBP,maturities,m,n)

We create a few data containers to facilitate the passing of data as input, and to facilitate the transformation of dates to time values.

In [ ]:
model={}
model["m13"]=m_USD_Euro
model["s13"]=s_USD_Euro
model["r13_0"]=Spot["EURUSD"][-1]
model["m21"]=m_GBP_USD
model["s21"]=s_GBP_USD
model["r21_0"]=1/Spot["GBPUSD"][-1]
model["m32"]=m_Euro_GBP
model["s32"]=s_Euro_GBP
model["r32_0"]=1/Spot["EURGBP"][-1]

t=to_workday(fund_dates[0])
T=to_workday(fund_dates[-1])
dat_to_mat={}
dat_to_mat[t]=0
dat_to_ind={}
dat_to_ind[t]=0
mat_to_ind={}
mat_to_ind[0]=0
ind_to_dat=[]
ind_to_dat.append(t)
i=1

while (t<to_workday(T)):
	dat_to_mat[next_workday(t)]=dat_to_mat[t]+1/252 #this means that we are off by ~1/252 per year; that is an acceptable deviation
	mat_to_ind[dat_to_mat[next_workday(t)]]=i
	t=next_workday(t)
	dat_to_ind[t]=i
	ind_to_dat.append(t)
	i=i+1

time_axis={}
time_axis["dtm"]=dat_to_mat
time_axis["mti"]=mat_to_ind
time_axis["itd"]=ind_to_dat
time_axis["dti"]=dat_to_ind

Now, we are ready to calculate the IRR distribution of our fund. First, we introduce a couple of functions to compute the IRR of a single flow, and then we sample $N$ realizations of the Brownian motions, calculate the IRR on each of those realizations, and use that to extract the distribution of the IRR and the IRR at the 5% level.

In [ ]:
def IRR_residual(x,R,T): 
	y=0
	n=len(T)
	for i in range(n):
		t=relativedelta(T[i],T[0])
		y=y+R[i]/x**(t.years+t.months/12+t.days/252)
	return y

#A function to retrieve the IRR of a deterministic cash flow, in a single currency.
def IRR(R,T):
	l=1
	r=2
	x=(l+r)/2
	tol=10**(-3)
	fl=IRR_residual(l,R,T)
	fr=IRR_residual(r,R,T)
	df=fl-fr
	while (df>tol):
		x=(l+r)/2
		fx=IRR_residual(x,R,T)
		if (fx>0):
			l=x
			fl=fx
			df=fl-fr
		if (fx<=0):
			r=x
			fr=fx
			df=fl-fr
	return x

#Sample a single path of the 2 Brownian motions associated with the exchange rates
def sample_path(model,time_axis):
	m13=model["m13"]
	s13=model["s13"]
	r13_0=model["r13_0"]
	m21=model["m21"]
	s21=model["s21"]
	r21_0=model["r21_0"]
	m32=model["m32"]
	s32=model["s32"]
	r32_0=model["r32_0"]
	
	#Some variables just introduced for convenience
	dtm=time_axis["dtm"]
	mti=time_axis["mti"]
	itd=time_axis["itd"]
	dti=time_axis["dti"]

	rho_13_21=(s32**2-s13**2-s21**2)/(2*s13*s21)

	N=len(dtm.keys())

	W1=np.zeros(N)
	Wh=np.zeros(N)
	W2=np.zeros(N)
	sample_13=np.zeros(N)
	norm_13=np.zeros(N)
	sample_21=np.zeros(N)
	
	#We could also write down the covariance matrices of the brownian motions and sample using Cholesky decomposition, but that would be slower
	for i in range(1,N):
	#Sample one realization of two independent Brownian motions
		W1[i]=W1[i-1]+np.sqrt(1/252)*np.random.normal()
		Wh[i]=Wh[i-1]+np.sqrt(1/252)*np.random.normal()
		#Extract a Brownian motion that is correlated with W1
		W2[i]=rho_13_21*W1[i]+np.sqrt(1-rho_13_21**2)*Wh[i]

	for i in range(N):
	#Extract the realization exchange rates from the realization of the Brownian motions
		sample_13[i]=r13_0*np.exp((m13-s13**2/2)*dtm[itd[i]])*np.exp(s13*W1[i])
		norm_13[i]=r13_0*np.exp(s13*W1[i])
		sample_21[i]=r21_0*np.exp((m21-s21**2/2)*dtm[itd[i]])*np.exp(s21*W2[i])
	return (sample_13,sample_21)

#A function to sample N values from the distribution of the IRR
def IRR_distribution(fund,model,N,time_axis):
	dates=fund['Date'].tolist()
	USD=fund['USD flow'].tolist()
	Euro=fund['Euro flow'].tolist()
	GBP=fund['GBP flow'].tolist()
	
	#We are just setting up some convenient names here
	dtm=time_axis["dtm"]
	mti=time_axis["mti"]
	itd=time_axis["itd"]
	dti=time_axis["dti"]
	
	sample_USD_Euro=[]
	sample_GBP_USD=[]
	irr=[]
	flow=[]
	for i in range(N):
	#At each iteration we sample a path, then we calculate the IRR on this path
		(sample1,sample2)=sample_path(model,time_axis)
		sample_USD_Euro.append(sample1)
		sample_GBP_USD.append(sample2)
		flow.append([])
		for j in range(len(dates)):
			d=to_workday(dates[j])
			flow[i].append(USD[j]+Euro[j]*sample_USD_Euro[i][dti[d]]+GBP[j]/sample_GBP_USD[i][dti[d]])
		irr.append(IRR(flow[i],dates))
		print(i+1,":",irr[i])
	
	return (irr,sample_USD_Euro,sample_GBP_USD)

N=1000
(irr,sample_USD_Euro,sample_GBP_USD)=IRR_distribution(fund_data,model,N,time_axis)

We are now able to plot a histogram of the IRR distribution; the dashed lines represent the 5%, 50% and 95% levels. 

In [ ]:
h=max(plt.hist(irr,50,density=True,color='y')[0])
x=np.linspace(min(irr)-0.05,max(irr)+0.05,1000)
(mu,var)=stats.norm.fit(irr)
y=stats.norm.pdf(x,mu,var)
plt.plot(x,y,color='b')
a=[sorted(irr)[int(N/20)],sorted(irr)[int(N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irr)[int(19*N/20)],sorted(irr)[int(19*N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irr)[int(N/2)],sorted(irr)[int(N/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.show()

print("IRR at risk-5%:",sorted(irr)[50])

We can conluce that the IRR at risk at the 5% level is approximately 9.7%. To ameliorate this, we will use barrier options. 

The payoff of barrier options is higly path-dependent. Therefore, using Monte Carlo simulation to approximate the prices of such options is among our better alternatives.

In [ ]:
#Monte Carlo evaluation of a barrier put option 
def evaluate_barrier_put_option(barrier,strike,r0,maturity,rd,rf,volatility,time_axis,N):
#Setting up some convenience variables
	dtm=time_axis["dtm"]
	mti=time_axis["mti"]
	itd=time_axis["itd"]
	dti=time_axis["dti"]
	
	V=0
	A=[]
	B=[]
	x=0
	activations=0
	for i in range(N): #We sample N paths
		t=0
		W=0
		activate=False
		while (t<maturity and activate==False):
			W=W+np.sqrt(1/252)*np.random.normal()
			R=r0*np.exp((rd-rf-volatility**2/2)*t)*np.exp(volatility*W)
			
			#For each path we check if the barrier is activated
			if (R<=barrier):
				activate=True
				activations=activations+1
			
			t=t+1/252
		
		#If the barrier is activated, add the discounted payoff to the value
		if activate==True:
			while (t<maturity):
				W=W+np.random.normal()*np.sqrt(1/252)
				t=t+1/252
			R=r0*np.exp((rd-rf-volatility**2/2)*t)*np.exp(volatility*W)
			V=V+max(0,strike-R)/N*np.exp(-rd*maturity)
	return V

#Same as above, but for call options
def evaluate_barrier_call_option(barrier,strike,r0,maturity,rd,rf,volatility,time_axis,N):
	dtm=time_axis["dtm"]
	mti=time_axis["mti"]
	itd=time_axis["itd"]
	dti=time_axis["dti"]
	
	V=0
	A=[]
	B=[]
	x=0
	activations=0
	for i in range(N):
		t=0
		W=0
		activate=False
		while (t<=maturity and activate==False):
			W=W+np.sqrt(1/252)*np.random.normal()
			R=r0*np.exp((rd-rf-volatility**2/2)*t)*np.exp(volatility*W)
			
			if (R>=barrier):
				activate=True
				activations=activations+1
			
			t=t+1/252
		
		if activate==True:
			while (t<maturity):
				W=W+np.random.normal()*np.sqrt(1/252)
				t=t+1/252
			R=r0*np.exp((rd-rf-volatility**2/2)*t)*np.exp(volatility*W)
			V=V+max(0,R-strike)/N*np.exp(-rd*maturity)
			
	return V

For an exchange rate $R(t)$ with drift $\mu$ and volatility $\sigma$, and a cash flow in our fund that comes in at time $T$, we use a pair of options: one with barrier 
$$B_1=R(0) e^{(\mu-\sigma^2/2)T}e^{-1.6\sigma \sqrt{T}}$$
and strike 
$$K_1=R(0) e^{(\mu-\sigma^2/2)T}e^{1.6\sigma \sqrt{T}},$$
and one with barrier 
$$B_2=R(0) e^{(\mu-\sigma^2/2)T}e^{+1.6\sigma \sqrt{T}}$$
and strike 
$$K_2=R(0) e^{(\mu-\sigma^2/2)T}e^{-1.6\sigma \sqrt{T}}.$$

We proceed to approximate the barrier prices; since we need to sample the exchange rate for every day of the time period a barrier option can be active (up to ~1250 days), and we sample 1000 paths for each option, and we need to calculate 4 option prices (1 call/1 put for 2 exchange rates) per individual cash flow (one cash flow per 3 months for 5 years), the next step might take a while.

In [ ]:
r_USD_Euro=model["r13_0"]
r_USD_GBP=1/model["r21_0"]

r_USD_Euro=Spot["EURUSD"][-1]
T=5



#We reverse the GBP-USD rate for convenience
m_USD_GBP=-m_GBP_USD
s_USD_GBP=s_GBP_USD

#A function to retrieve the prices of a pair of options
def get_barriers(r1,r2,r12,m,s,q,fund_dates,time_axis,N):
	bput=[]
	bcall=[]
	for d in fund_dates[1:]:
	#Here we just calculate the value of each option, if the nominal value was 1

		T=time_axis['dtm'][to_workday(d)]
		print("maturity:",T)
		
		barrier=r12*np.exp((m-s**2/2)*T)*np.exp(-q*s*np.sqrt(T))
		strike=r12*np.exp((m-s**2/2)*T)*np.exp(q*s*np.sqrt(T))
		x=evaluate_barrier_put_option(barrier,strike,r12,T,r1,r2,s,time_axis,N)
		print("Put value:",x)
		bput.append({"maturity":T, "barrier":barrier, "strike":strike, "value":x})
		
		barrier=r12*np.exp((m-s**2/2)*T)*np.exp(q*s*np.sqrt(T))
		strike=r12*np.exp((m-s**2/2)*T)*np.exp(-q*s*np.sqrt(T))
		x=evaluate_barrier_call_option(barrier,strike,r12,T,r1,r2,s,time_axis,N)
		print("Call value:",x)
		bcall.append({"maturity":T, "barrier":barrier, "strike":strike, "value":x})

		print()
	return (bput,bcall)

    
#Calculating barrier prices
barriers={}
q=1.6 #Strike and barrier are set to q times std. deviation
print("Calculating USD-Euro barrier prices")
(barriers["Euro put"],barriers["Euro call"])=get_barriers(r_USD,r_Euro,r_USD_Euro,m_USD_Euro,s_USD_Euro,q,fund_dates,time_axis,N)
print("Calculating USD-GBP barrier prices")
(barriers["GBP put"],barriers["GBP call"])=get_barriers(r_USD,r_GBP,r_USD_GBP,m_USD_GBP,s_USD_GBP,q,fund_dates,time_axis,N)
#Save prices
bars={}
bars[q]=barriers

Observe that the activation probabilities are not always close to 5%. This is due to the fact that the barriers are chosen to be the exchange rates at the 5% level at the time of expiry of each option, so the probabilty that the exchange rate crosses this barrier any time before that is higher than 5%.

Using the prices we calculated, we are able to determine how much of each option we need to buy and sell in order to ensure that our hedging strategy is self-financing. Then, we can sample the IRR distribution after hedging.

In [ ]:
#A function to sample N values from the distribution of the IRR, after having incorporated the barrier options
def IRR_distribution_hedged(fund,model,barriers,N,time_axis,nominal_ratio):
	dates=fund['Date'].tolist()
	USD=fund['USD flow'].tolist()
	Euro=fund['Euro flow'].tolist()
	GBP=fund['GBP flow'].tolist()
	
	dtm=time_axis["dtm"]
	mti=time_axis["mti"]
	itd=time_axis["itd"]
	dti=time_axis["dti"]
	
	#Choosing the correct nominal values, so that the hedging strategy is self-financing
	for i in range(len(barriers["Euro call"])):
		if (barriers["Euro put"][i]["value"]<=barriers["Euro call"][i]["value"]):
			barriers["Euro put"][i]["nominal"]=Euro[i+1]/nominal_ratio
			barriers["Euro call"][i]["nominal"]=barriers["Euro put"][i]["nominal"]*barriers["Euro put"][i]["value"]/barriers["Euro call"][i]["value"]
		else:
			barriers["Euro call"][i]["nominal"]=Euro[i+1]/nominal_ratio
			barriers["Euro put"][i]["nominal"]=barriers["Euro call"][i]["nominal"]*barriers["Euro call"][i]["value"]/barriers["Euro put"][i]["value"]
		
		if (barriers["GBP put"][i]["value"]<=barriers["GBP call"][i]["value"]):
			barriers["GBP put"][i]["nominal"]=GBP[i+1]/nominal_ratio
			barriers["GBP call"][i]["nominal"]=barriers["GBP put"][i]["nominal"]*barriers["GBP put"][i]["value"]/barriers["GBP call"][i]["value"]
		else:
			barriers["GBP call"][i]["nominal"]=GBP[i+1]/nominal_ratio
			barriers["GBP put"][i]["nominal"]=barriers["GBP call"][i]["nominal"]*barriers["GBP call"][i]["value"]/barriers["GBP put"][i]["value"]
		

	
	sample_USD_Euro=[]
	sample_GBP_USD=[]
	irr=[]
	flow=[]
	for i in range(N):
	#At each iteration we sample a path, then we calculate the IRR on this path
		(sample1,sample2)=sample_path(model,time_axis)
		sample_USD_Euro.append(sample1)
		sample_GBP_USD.append(sample2)
		flow.append([])
		
		activate_put={}
		activate_call={}
		for b in barriers["Euro put"]:
			activate_put[b["maturity"]]=False
			
		for b in barriers["Euro call"]:
			activate_call[b["maturity"]]=False
			
		for b in barriers["GBP put"]:
			activate_put[b["maturity"]]=False
			
		for b in barriers["GBP call"]:
			activate_call[b["maturity"]]=False
		
		#Go through the path to see which barriers are activated
		for j in range(len(itd)):
			t=dtm[itd[j]]
			for b in barriers["Euro put"]:
				if sample1[j]<=b["barrier"]:
					activate_put[b["maturity"]]=True
			for b in barriers["Euro call"]:
				if sample1[j]>=b["barrier"]:
					activate_call[b["maturity"]]=True
			for b in barriers["GBP put"]:
				if sample1[j]<=b["barrier"]:
					activate_put[b["maturity"]]=True
			for b in barriers["GBP call"]:
				if sample1[j]>=b["barrier"]:
					activate_call[b["maturity"]]=True
		
		
		for j in range(len(dates)):
			d=to_workday(dates[j])
			flow[i].append(USD[j]+GBP[j]/sample_GBP_USD[i][dti[d]]+Euro[j]*sample_USD_Euro[i][dti[d]])
			
			#Add the cash flow of the activated barriers
			for b in barriers["Euro put"]:
				if b["maturity"]==dtm[d] and activate_put[b["maturity"]]==True:
					R=sample_USD_Euro[i][dti[d]]
					flow[i][-1]=flow[i][-1]+b["nominal"]*max(0,b["strike"]-R)
					
			for b in barriers["Euro call"]:
				if b["maturity"]==dtm[d] and activate_call[b["maturity"]]==True:
					R=sample_USD_Euro[i][dti[d]]
					flow[i][-1]=flow[i][-1]-b["nominal"]*max(0,R-b["strike"])
			
			for b in barriers["GBP put"]:
				if b["maturity"]==dtm[d] and activate_put[b["maturity"]]==True:
					R=1/sample_GBP_USD[i][dti[d]]
					flow[i][-1]=flow[i][-1]+b["nominal"]*max(0,b["strike"]-R)
					
			for b in barriers["GBP call"]:
				if b["maturity"]==dtm[d] and activate_call[b["maturity"]]==True:
					R=1/sample_GBP_USD[i][dti[d]]
					flow[i][-1]=flow[i][-1]-b["nominal"]*max(0,R-b["strike"])
					
		irr.append(IRR(flow[i],dates))
		print(i+1,":",irr[i])
	
	return (irr,sample_USD_Euro,sample_GBP_USD)

(irh,sample_USD_Euro,sample_GBP_USD)=IRR_distribution_hedged(fund_data,model,barriers,N,time_axis,2)

In [ ]:
h=max(plt.hist(irh,30,density=True,color='y')[0])
x=np.linspace(min(irh)-0.05,max(irh)+0.05,1000)
a=[sorted(irh)[int(N/20)],sorted(irh)[int(N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(19*N/20)],sorted(irh)[int(19*N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(N/2)],sorted(irh)[int(N/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.show()

print("IRR at risk-5%:",sorted(irh)[50])

We get an IRR at risk at the 5% level of ~12.7%, which constitutes an increase of 3%. The mean IRR is also (very) marginally improved, since the IRR is concave at the cash flows, and higher concentration around the mean of the cash flows increases the mean IRR. Still, the difference is very small.

We observe that the tail of the distribution is lighter than before, but still maybe a bit too heavy. We try a new set of barriers and strikes.

In [ ]:
#Calculating barrier prices
barriers={}
q=1.3 #Strike and barrier are set to q times std. deviation
print("Calculating USD-Euro barrier prices")
(barriers["Euro put"],barriers["Euro call"])=get_barriers(r_USD,r_Euro,r_USD_Euro,m_USD_Euro,s_USD_Euro,q,fund_dates,time_axis,N)
print("Calculating USD-GBP barrier prices")
(barriers["GBP put"],barriers["GBP call"])=get_barriers(r_USD,r_GBP,r_USD_GBP,m_USD_GBP,s_USD_GBP,q,fund_dates,time_axis,N)
#Save prices
bars={}
bars[q]=barriers

In [ ]:
(irh,sample_USD_Euro,sample_GBP_USD)=IRR_distribution_hedged(fund_data,model,barriers,N,time_axis,2)

In [ ]:
h=max(plt.hist(irh,30,density=True,color='y')[0])
x=np.linspace(min(irh)-0.05,max(irh)+0.05,1000)
a=[sorted(irh)[int(N/20)],sorted(irh)[int(N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(19*N/20)],sorted(irh)[int(19*N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(N/2)],sorted(irh)[int(N/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.show()

print("IRR at risk-5%:",sorted(irh)[50])

We go from 12.7% IRR at the 5% level to 13.4%; we should be close to the limits of our approach.

In [ ]:
#Calculating barrier prices
barriers={}
q=0.7 #Strike and barrier are set to q times std. deviation
print("Calculating USD-Euro barrier prices")
(barriers["Euro put"],barriers["Euro call"])=get_barriers(r_USD,r_Euro,r_USD_Euro,m_USD_Euro,s_USD_Euro,q,fund_dates,time_axis,N)
print("Calculating USD-GBP barrier prices")
(barriers["GBP put"],barriers["GBP call"])=get_barriers(r_USD,r_GBP,r_USD_GBP,m_USD_GBP,s_USD_GBP,q,fund_dates,time_axis,N)
#Save prices
bars[q]=barriers

In [ ]:
(irh,sample_USD_Euro,sample_GBP_USD)=IRR_distribution_hedged(fund_data,model,barriers,N,time_axis,2)

In [ ]:
h=max(plt.hist(irh,30,density=True,color='y')[0])
x=np.linspace(min(irh)-0.05,max(irh)+0.05,1000)
a=[sorted(irh)[int(N/20)],sorted(irh)[int(N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(19*N/20)],sorted(irh)[int(19*N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(N/2)],sorted(irh)[int(N/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.show()

print("IRR at risk-5%:",sorted(irh)[50])

The improvement is very marginal, from 13.4% to 13.5%.

We have tried barriers and strikes based around $1.6\sigma$, $1.3\sigma$ and $0.7\sigma$, which are essentially the 5%/10%/25% levels of a normal distribution. The idea was that if we get a value that falls in the bottom $x\%$, we balance it out by adding a value that falls in the top $x\%$, thus falling somewhere in the $[x\%,(100-x)\%]$ region. As a result, we do not expect this strategy to work if we choose barriers and strikes corresponding to $x\%$ with $25<x\leq 50$, since (intuitively) the regions from which we gain and the regions from which we lose start to overlap.

In [ ]:
#Calculating barrier prices
barriers={}
q=0.4 #Strike and barrier are set to q times std. deviation
print("Calculating USD-Euro barrier prices")
(barriers["Euro put"],barriers["Euro call"])=get_barriers(r_USD,r_Euro,r_USD_Euro,m_USD_Euro,s_USD_Euro,q,fund_dates,time_axis,N)
print("Calculating USD-GBP barrier prices")
(barriers["GBP put"],barriers["GBP call"])=get_barriers(r_USD,r_GBP,r_USD_GBP,m_USD_GBP,s_USD_GBP,q,fund_dates,time_axis,N)
#Save prices
bars[q]=barriers

In [ ]:
(irh,sample_USD_Euro,sample_GBP_USD)=IRR_distribution_hedged(fund_data,model,barriers,N,time_axis,2)

In [ ]:
h=max(plt.hist(irh,30,density=True,color='y')[0])
x=np.linspace(min(irh)-0.05,max(irh)+0.05,1000)
a=[sorted(irh)[int(N/20)],sorted(irh)[int(N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(19*N/20)],sorted(irh)[int(19*N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(N/2)],sorted(irh)[int(N/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.show()

print("IRR at risk-5%:",sorted(irh)[50])

As was predicted, going too far will start having adverse effects on the IRR. Hence, we stick to the barriers/strikes based around $0.7\sigma$, which gave an IRR at the 5% level of 13.5%

In [ ]:
(irh,sample_USD_Euro,sample_GBP_USD)=IRR_distribution_hedged(fund_data,model,bars[0.7],N,time_axis,1.6)

In [ ]:
h=max(plt.hist(irh,30,density=True,color='y')[0])
x=np.linspace(min(irh)-0.05,max(irh)+0.05,1000)
a=[sorted(irh)[int(N/20)],sorted(irh)[int(N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(19*N/20)],sorted(irh)[int(19*N/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(N/2)],sorted(irh)[int(N/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.show()

print("IRR at risk-5%:",sorted(irh)[50])

For nominal values equal to the respective cash flow over 1.6, we found the best IRR at the 5% level of 14.4%; this constitutes an improvement of ~4.7%. over the unhedged IRR, approximately 1% below the mean IRR.

To conclude, under the present model we saw an initial IRR at risk at the 5% level of ~9.7%. Using the best parameters we could find for the barrier options we used, we pushed it to 14.4%, while the best IRR achievable is ~15.3%. We conjecture that with more complicated options, e.g. with options that have 2 activation conditions instead of 1, we could get much closer to the best possible IRR.

Finally, as an example, we apply our model to the existing data with start date of 1/1/2011. Since each calibration and evaluation takes a few minutes, it would require a substantial amount of time to produce a number of experiments that have any statistical importance. 

In [ ]:
import datetime

def calibrate_vol(C,maturities,rd,rf,guess):
	(m,n)=np.shape(C)
	x = tf.Variable(guess, name='x', dtype=tf.float32)
	E=0

	for i in range(m):
		for j in range(n):
			T=maturities[i]
			d2=(rd-rf-x**2/2)*T/(x*np.sqrt(T))
			d1=d2+x*np.sqrt(T)
			price=tfp.distributions.Normal(0,1).cdf(d1)*np.exp(-rf*T)-tfp.distributions.Normal(0,1).cdf(d2)*np.exp(-rd*T) #Predicted option price
			E=E+(C[i][j]-price)**2/(m*n)#Squared error between historic and predicted price

#Having set up the error function, tensorflow takes care of differentiating and optimizing
	optimizer = tf.train.GradientDescentOptimizer(0.35)
	train = optimizer.minimize(E)
	init = tf.initialize_all_variables()
	def optimize():
		with tf.Session() as session:
			session.run(init)
			fx=session.run(E)
			fprev=0
			print("starting at", "x:", session.run(x), "MSE:", fx)
			step=0
			while np.abs(fx-fprev)>=10**(-10):
				fprev=fx
				session.run(train)
				fx=session.run(E)
				print("step", step, "x:", session.run(x), "MSE:", fx)
				step=step+1
			ret=session.run(x)
			session.close()
			return ret

	x_opt=optimize()
	return x_opt

irr=[]
irh=[]

start=market_dates[-2151]
while start<datetime.datetime(2014,1,1,0,0):
	D=to_workday(start)
	Di=market_dates.index(D)
	print(D)

	def sample_prices(imp_vol,maturities,rd,rf,m,n):
			S=np.zeros(shape=(m,n))
			C=np.zeros(shape=(m,n))
			maturities=list(imp_vol.keys())
			for i in range(m):
				for j in range(n):
					S[i][j]=imp_vol[maturities[i]][Di-j]
					C[i][j]=vanilla_atm_call_option(rd,rf,S[i][j]/100,maturities[i])
			return (C,S)
		
	#We just use the existing code, plugging in the appropriate dates, and using the historical rates instead of simulated ones.
	N=1000
	r_USD=np.log((1+Yield["USD"]["3M"][Di]/100))
	r_Euro=np.log((1+Yield["EUR"]["3M"][Di]/100))
	r_GBP=np.log((1+Yield["GBP"]["3M"][Di]/100))
	print("USD short rate: ",r_USD)
	print("Euro short rate: ",r_Euro)
	print("GBP short rate: ",r_GBP)

	print("\nX-Y spot rate: how much of X one unit of Y can buy")
	m_USD_Euro=r_USD-r_Euro
	m_GBP_USD=r_GBP-r_USD
	m_Euro_GBP=r_Euro-r_GBP

	print("USD-Euro drift: ",m_USD_Euro)
	print("USD-Euro spot rate:",Spot["EURUSD"][Di])

	print("GBP-USD drift: ",m_GBP_USD)
	print("GBP-USD spot rate:",1/Spot["GBPUSD"][Di])

	print("Euro-GBP drift: ",m_Euro_GBP)
	print("Euro-GBP spot rate:",1/Spot["EURGBP"][Di])

	n=1 #We calibrate with respect to the prices observed in the n last days
	maturities=list(Imp_vol['EURUSD'].keys())
	m=len(maturities)-2 #We consider options with m different maturities; essentially, we disregard the 7y and 10y maturities. We have chosen a constant volatility model, but the volatilities of these maturities are so different than the rest of the data, that their inclusion will more likely contribute to our model being more inaccurate.

	#We calibrate the volatilities of the 3 exchange rates
	print("Calibrating USD-Euro volatility to minimize MSE of option prices in last ",n," day(s)")
	(C_USD_Euro,S_USD_Euro)=sample_prices(Imp_vol['EURUSD'],maturities,r_USD,r_Euro,m,n)
	s_USD_Euro=calibrate_vol(C_USD_Euro,maturities,r_USD,r_Euro,Imp_vol['EURUSD'][1/252][Di])

	print("Calibrating GBP-USD volatility to minimize MSE of option prices in last ",n," day(s)")
	(C_GBP_USD,S_GBP_USD)=sample_prices(Imp_vol['GBPUSD'],maturities,r_GBP,r_USD,m,n)
	s_GBP_USD=calibrate_vol(C_GBP_USD,maturities,r_GBP,r_USD,Imp_vol['GBPUSD'][1/252][Di])

	print("Calibrating Euro-GBP volatility to minimize MSE of option prices in last ",n," day(s)")
	(C_Euro_GBP,S_GBP_USD)=sample_prices(Imp_vol['EURGBP'],maturities,r_Euro,r_GBP,m,n)
	s_Euro_GBP=calibrate_vol(C_Euro_GBP,maturities,r_Euro,r_GBP,Imp_vol['EURGBP'][1/252][Di])

	#We create a set of dictionaries whose only purpose is to have a convenient way of translating dates to how much time since time 0 passed, and vice versa
	#In an ideal world we would only have time as a variable and not dates
	model={}
	model["m13"]=m_USD_Euro
	model["s13"]=s_USD_Euro
	model["r13_0"]=Spot["EURUSD"][Di]
	model["m21"]=m_GBP_USD
	model["s21"]=s_GBP_USD
	model["r21_0"]=1/Spot["GBPUSD"][Di]
	model["m32"]=m_Euro_GBP
	model["s32"]=s_Euro_GBP
	model["r32_0"]=1/Spot["EURGBP"][Di]

	DT=market_dates[Di]-market_dates[-1]
	t=to_workday(fund_dates[0])+DT
	T=to_workday(fund_dates[-1])+DT
	dat_to_mat={}
	dat_to_mat[t]=0
	dat_to_ind={}
	dat_to_ind[t]=0
	mat_to_ind={}
	mat_to_ind[0]=0
	ind_to_dat=[]
	ind_to_dat.append(t)
	i=1

	while (t<to_workday(T)):
		dat_to_mat[next_workday(t)]=dat_to_mat[t]+1/252 #this means that we are off by ~1/252 per year; that is an acceptable deviation
		mat_to_ind[dat_to_mat[next_workday(t)]]=i
		t=next_workday(t)
		dat_to_ind[t]=i
		ind_to_dat.append(t)
		i=i+1

	time_axis={}
	time_axis["dtm"]=dat_to_mat
	time_axis["mti"]=mat_to_ind
	time_axis["itd"]=ind_to_dat
	time_axis["dti"]=dat_to_ind

	#Calculate the IRR before hedging
	dates=fund_data['Date'].tolist()
	USD=fund_data['USD flow'].tolist()
	Euro=fund_data['Euro flow'].tolist()
	GBP=fund_data['GBP flow'].tolist()
		
	dtm=time_axis["dtm"]
	mti=time_axis["mti"]
	itd=time_axis["itd"]
	dti=time_axis["dti"]

	for i in range(len(fund_dates)):
		dates[i]=dates[i]+DT
		
	flow=[]
		
	for j in range(len(dates)):
		d=to_workday(dates[j])
		gu=market_data.loc[market_data['Date']==d]['Spot GBPUSD'].values[0]
		eu=market_data.loc[market_data['Date']==d]['Spot EURUSD'].values[0]
		flow.append(USD[j]+Euro[j]*eu+GBP[j]*gu)
	print("IRR before hedging:",IRR(flow,dates))
	irr.append(IRR(flow,dates))

	#Next we create the barrier options
	r_USD_Euro=model["r13_0"]
	r_USD_GBP=1/model["r21_0"]

	r_USD_Euro=Spot["EURUSD"][Di]
	T=5

	#We reverse the GBP-USD rate for convenience
	m_USD_GBP=-m_GBP_USD

	#Calculating barrier prices
	barriers={}
	print("Calculating USD-Euro barrier prices")
	(barriers["Euro put"],barriers["Euro call"])=get_barriers(r_USD,r_Euro,r_USD_Euro,m_USD_Euro,s_USD_Euro,0.7,dates,time_axis,N)
	print("Calculating USD-GBP barrier prices")
	(barriers["GBP put"],barriers["GBP call"])=get_barriers(r_USD,r_GBP,r_USD_GBP,m_USD_GBP,s_USD_GBP,0.7,dates,time_axis,N)

	#Choosing the correct nominal values, so that the hedging strategy is self-financing
	for i in range(len(barriers["Euro call"])):
		if (barriers["Euro put"][i]["value"]<=barriers["Euro call"][i]["value"]):
			barriers["Euro put"][i]["nominal"]=Euro[i+1]/1.6
			barriers["Euro call"][i]["nominal"]=barriers["Euro put"][i]["nominal"]*barriers["Euro put"][i]["value"]/barriers["Euro call"][i]["value"]
		else:
			barriers["Euro call"][i]["nominal"]=Euro[i+1]/1.6
			barriers["Euro put"][i]["nominal"]=barriers["Euro call"][i]["nominal"]*barriers["Euro call"][i]["value"]/barriers["Euro put"][i]["value"]
		
		if (barriers["GBP put"][i]["value"]<=barriers["GBP call"][i]["value"]):
			barriers["GBP put"][i]["nominal"]=GBP[i+1]/1.6
			barriers["GBP call"][i]["nominal"]=barriers["GBP put"][i]["nominal"]*barriers["GBP put"][i]["value"]/barriers["GBP call"][i]["value"]
		else:
			barriers["GBP call"][i]["nominal"]=GBP[i+1]/1.6
			barriers["GBP put"][i]["nominal"]=barriers["GBP call"][i]["nominal"]*barriers["GBP call"][i]["value"]/barriers["GBP put"][i]["value"]

	activate_put={}
	activate_call={}
	for b in barriers["Euro put"]:
		activate_put[b["maturity"]]=False
		
	for b in barriers["Euro call"]:
		activate_call[b["maturity"]]=False
		
	for b in barriers["GBP put"]:
		activate_put[b["maturity"]]=False
		
	for b in barriers["GBP call"]:
		activate_call[b["maturity"]]=False

	#Go through the path to see which barriers are activated
	for j in range(len(itd)):
		gu=market_data.loc[market_data['Date']==d]['Spot GBPUSD'].values[0]
		eu=market_data.loc[market_data['Date']==d]['Spot EURUSD'].values[0]
		t=dtm[itd[j]]
		for b in barriers["Euro put"]:
			if eu<=b["barrier"]:
				activate_put[b["maturity"]]=True
		for b in barriers["Euro call"]:
			if eu>=b["barrier"]:
				activate_call[b["maturity"]]=True
		for b in barriers["GBP put"]:
			if gu<=b["barrier"]:
				activate_put[b["maturity"]]=True
		for b in barriers["GBP call"]:
			if gu>=b["barrier"]:
				activate_call[b["maturity"]]=True
	flow=[]
	for j in range(len(dates)):
		d=to_workday(dates[j])
		gu=market_data.loc[market_data['Date']==d]['Spot GBPUSD'].values[0]
		eu=market_data.loc[market_data['Date']==d]['Spot EURUSD'].values[0]
		flow.append(0)
		#Add the cash flow of the activated barriers
		for b in barriers["Euro put"]:
			if b["maturity"]==dtm[d] and activate_put[b["maturity"]]==True:
				R=eu
				flow[-1]=flow[-1]+b["nominal"]*max(0,b["strike"]-R)
				
		for b in barriers["Euro call"]:
			if b["maturity"]==dtm[d] and activate_call[b["maturity"]]==True:
				R=eu
				flow[-1]=flow[-1]-b["nominal"]*max(0,R-b["strike"])
		
		for b in barriers["GBP put"]:
			if b["maturity"]==dtm[d] and activate_put[b["maturity"]]==True:
				R=gu
				flow[-1]=flow[-1]+b["nominal"]*max(0,b["strike"]-R)
				
		for b in barriers["GBP call"]:
			if b["maturity"]==dtm[d] and activate_call[b["maturity"]]==True:
				R=gu
				flow[-1]=flow[-1]-b["nominal"]*max(0,R-b["strike"])
		flow[-1]=flow[-1]+(USD[j]+Euro[j]*eu+GBP[j]*gu)
	print("IRR after hedging:",IRR(flow,dates))
	irh.append(IRR(flow,dates))
	
	start=start+relativedelta(months=3)


In [ ]:
n=len(irr)
h=max(plt.hist(irr,density=True,color='y')[0])
x=np.linspace(min(irr)-0.05,max(irr)+0.05,1000)
a=[sorted(irr)[int(n/20)],sorted(irr)[int(n/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irr)[int(19*n/20)],sorted(irr)[int(19*n/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irr)[int(n/2)],sorted(irr)[int(n/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.title('IRR before hedging')
plt.show()

n=len(irh)
h=max(plt.hist(irh,density=True,color='y')[0])
x=np.linspace(min(irh)-0.05,max(irh)+0.05,1000)
a=[sorted(irh)[int(n/20)],sorted(irh)[int(n/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(19*n/20)],sorted(irh)[int(19*n/20)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
a=[sorted(irh)[int(n/2)],sorted(irh)[int(n/2)]]
b=[0,h+1]
plt.plot(a,b,linestyle='dashed',color='r')
plt.title('IRR after hedging')
plt.show()

print(irr)
print(irh)
plt.plot(np.array(irh)-np.array(irr))
plt.title('IRR after hedging minus IRR before hedging')
plt.show()